# E-Commerce Behavior Analysis
### BUDT737 Big Data and Artificial Intelligence for Business

We use a dataset of multicategory online stores to perform Exploratory Data Analysis and Predictive Modeling. Each row of the dataset is a transaction, indicating whether the user purchased the item at the end of the user journey. The analysis was performed by utilizing PySpark, AWS S3 and Databricks to produce efficient data processing workflow and drawing actionable insights for online stores to optimize their marketing strategies.

This project is about consumer behavior analysis. Consumer behavior analysis is a data-driven observation of online consumers and how they interact with the company; in this project, we have tried to understand and analyze how users interact with online stores.


**Business Questions:**
1. What is the most consumed product?
2. What is the most consumed category of all?
3. What are the top 5 brands that generate the most revenue?
4. What is the conversion rate of all customers for the brands?
5. Based on the given attributes in the dataset, how confident can we be in the prediction of  whether the users will purchase the product?

In [0]:
%python
#importing libraries
import pandas as pd
import numpy as np
import datetime as dt
import matplotlib.pyplot as plt
import seaborn as sns
from pyspark.sql.functions import col,isnan,when,count
from pyspark.ml.feature import OneHotEncoder,StringIndexer, VectorAssembler
from pyspark.ml import Pipeline
from pyspark.ml.classification import LogisticRegression, DecisionTreeClassifier
from pyspark.ml.evaluation import MulticlassClassificationEvaluator, BinaryClassificationEvaluator

### 1. Importing the data
The data is uploaded into AWS S3 buckets. We use Databricks to connect our dataset stored in AWS S3 buckets to the analysis platform. First, create a new user in AWS IAM and then set up the user to have full access to AWS S3. The reason to do this is because we need a user with credentials to mount S3 bucket to our notebook in Databricks and this user provides the credentials needed.

In [0]:
%scala
val AccessKey = "####################"
val SecretKey = "t##########+QRrlwZ9fz/N/0kVQRb9WcnxwN3i".replace("/", "%2F")
val AwsBucketName = "ecommercedatabudt737"
val MountName = "s3data2"

dbutils.fs.mount(s"s3a://$AccessKey:$SecretKey@$AwsBucketName", s"/mnt/$MountName")
display(dbutils.fs.ls(s"/mnt/$MountName"))

java.lang.IllegalArgumentException: requirement failed: Directory already mounted: /mnt/s3data2
	at com.databricks.backend.daemon.data.client.DbfsClient.send0(DbfsClient.scala:129)
	at com.databricks.backend.daemon.data.client.DbfsClient.sendIdempotent(DbfsClient.scala:69)
	at com.databricks.backend.daemon.dbutils.DBUtilsCore.createOrUpdateMount(DBUtilsCore.scala:758)
	at com.databricks.backend.daemon.dbutils.DBUtilsCore.mount(DBUtilsCore.scala:778)
	at com.databricks.dbutils_v1.impl.DbfsUtilsImpl.$anonfun$mount$1(DbfsUtilsImpl.scala:207)
	at scala.runtime.java8.JFunction0$mcZ$sp.apply(JFunction0$mcZ$sp.java:23)
	at com.databricks.logging.UsageLogging.$anonfun$recordOperation$1(UsageLogging.scala:413)
	at com.databricks.logging.UsageLogging.executeThunkAndCaptureResultTags$1(UsageLogging.scala:507)
	at com.databricks.logging.UsageLogging.$anonfun$recordOperationWithResultTags$4(UsageLogging.scala:528)
	at com.databricks.logging.Log4jUsageLoggingShim$.$anonfun$withAttributionContext$1(Log4jUsageLoggingShim.scala:32)
	at scala.util.DynamicVariable.withValue(DynamicVariable.scala:62)
	at com.databricks.logging.AttributionContext$.withValue(AttributionContext.scala:94)
	at com.databricks.logging.Log4jUsageLoggingShim$.withAttributionContext(Log4jUsageLoggingShim.scala:30)
	at com.databricks.logging.UsageLogging.withAttributionContext(UsageLogging.scala:283)
	at com.databricks.logging.UsageLogging.withAttributionContext$(UsageLogging.scala:282)
	at com.databricks.dbutils_v1.impl.DbfsUtilsImpl.withAttributionContext(DbfsUtilsImpl.scala:20)
	at com.databricks.logging.UsageLogging.withAttributionTags(UsageLogging.scala:318)
	at com.databricks.logging.UsageLogging.withAttributionTags$(UsageLogging.scala:303)
	at com.databricks.dbutils_v1.impl.DbfsUtilsImpl.withAttributionTags(DbfsUtilsImpl.scala:20)
	at com.databricks.logging.UsageLogging.recordOperationWithResultTags(UsageLogging.scala:502)
	at com.databricks.logging.UsageLogging.recordOperationWithResultTags$(UsageLogging.scala:422)
	at com.databricks.dbutils_v1.impl.DbfsUtilsImpl.recordOperationWithResultTags(DbfsUtilsImpl.scala:20)
	at com.databricks.logging.UsageLogging.recordOperation(UsageLogging.scala:413)
	at com.databricks.logging.UsageLogging.recordOperation$(UsageLogging.scala:385)
	at com.databricks.dbutils_v1.impl.DbfsUtilsImpl.recordOperation(DbfsUtilsImpl.scala:20)
	at com.databricks.dbutils_v1.impl.DbfsUtilsImpl.recordDbutilsFsOp(DbfsUtilsImpl.scala:94)
	at com.databricks.dbutils_v1.impl.DbfsUtilsImpl.mount(DbfsUtilsImpl.scala:208)
	at $line886d56449681485e91adff7f940c1af831.$read$$iw$$iw$$iw$$iw$$iw$$iw.<init>(command-2961455995355510:6)
	at $line886d56449681485e91adff7f940c1af831.$read$$iw$$iw$$iw$$iw$$iw.<init>(command-2961455995355510:54)
	at $line886d56449681485e91adff7f940c1af831.$read$$iw$$iw$$iw$$iw.<init>(command-2961455995355510:56)
	at $line886d56449681485e91adff7f940c1af831.$read$$iw$$iw$$iw.<init>(command-2961455995355510:58)
	at $line886d56449681485e91adff7f940c1af831.$read$$iw$$iw.<init>(command-2961455995355510:60)
	at $line886d56449681485e91adff7f940c1af831.$read$$iw.<init>(command-2961455995355510:62)
	at $line886d56449681485e91adff7f940c1af831.$read.<init>(command-2961455995355510:64)
	at $line886d56449681485e91adff7f940c1af831.$read$.<init>(command-2961455995355510:68)
	at $line886d56449681485e91adff7f940c1af831.$read$.<clinit>(command-2961455995355510)
	at $line886d56449681485e91adff7f940c1af831.$eval$.$print$lzycompute(<notebook>:7)
	at $line886d56449681485e91adff7f940c1af831.$eval$.$print(<notebook>:6)
	at $line886d56449681485e91adff7f940c1af831.$eval.$print(<notebook>)
	at sun.reflect.NativeMethodAccessorImpl.invoke0(Native Method)
	at sun.reflect.NativeMethodAccessorImpl.invoke(NativeMethodAccessorImpl.java:62)
	at sun.reflect.DelegatingMethodAccessorImpl.invoke(DelegatingMethodAccessorImpl.java:43)
	at java.lang.reflect.Method.invoke(Method.java:498)
	at scala.tools.nsc.interpreter.IMain$ReadEvalPrint.call(IMain.scala:747)
	at scala.tools.nsc.

In [0]:
# File location and type
file_location = "/mnt/s3data2/subset_data_2.csv"
file_type = "csv"

# CSV options
infer_schema = "true"
first_row_is_header = "true"
delimiter = ","

# The applied options are for CSV files. For other file types, these will be ignored.
df1 = spark.read.format(file_type) \
  .option("inferSchema", infer_schema) \
  .option("header", first_row_is_header) \
  .option("sep", delimiter) \
  .load(file_location)

display(df1)

_c0,event_time,event_type,product_id,category_id,category_code,brand,price,user_id,user_session
0,2019-10-01T00:00:00.000+0000,view,3900821,2053013552326770905,appliances.environment.water_heater,aqua,33.2,554748717,9333dfbd-b87a-4708-9857-6336556b0fcc
1,2019-10-01T00:00:01.000+0000,view,17200506,2053013559792632471,furniture.living_room.sofa,null,543.1,519107250,566511c2-e2e3-422b-b695-cf8e6e792ca8
2,2019-10-01T00:00:01.000+0000,view,1307067,2053013558920217191,computers.notebook,lenovo,251.74,550050854,7c90fc70-0e80-4590-96f3-13c02c18c713
3,2019-10-01T00:00:04.000+0000,view,1004237,2053013555631882655,electronics.smartphone,apple,1081.98,535871217,c6bd7419-2748-4c56-95b4-8cec9ff8b80d
4,2019-10-01T00:00:05.000+0000,view,1480613,2053013561092866779,computers.desktop,pulser,908.62,512742880,0d0d91c2-c9c2-4e81-90a5-86594dec0db9
5,2019-10-01T00:00:08.000+0000,view,17300353,2053013553853497655,null,creed,380.96,555447699,4fe811e9-91de-46da-90c3-bbd87ed3a65d
6,2019-10-01T00:00:08.000+0000,view,31500053,2053013558031024687,null,luminarc,41.16,550978835,6280d577-25c8-4147-99a7-abc6048498d6
7,2019-10-01T00:00:10.000+0000,view,28719074,2053013565480109009,apparel.shoes.keds,baden,102.71,520571932,ac1cd4e5-a3ce-4224-a2d7-ff660a105880
8,2019-10-01T00:00:11.000+0000,view,1004545,2053013555631882655,electronics.smartphone,huawei,566.01,537918940,406c46ed-90a4-4787-a43b-59a410c1a5fb
9,2019-10-01T00:00:11.000+0000,view,2900536,2053013554776244595,appliances.kitchen.microwave,elenberg,51.46,555158050,b5bdd0b3-4ca2-4c55-939e-9ce44bb50abd


In [0]:
# File location and type
file_location = "/mnt/s3data2/df_underover2.csv"
file_type = "csv"

# CSV options
infer_schema = "true"
first_row_is_header = "true"
delimiter = ","

# The applied options are for CSV files. For other file types, these will be ignored.
df_ml = spark.read.format(file_type) \
  .option("inferSchema", infer_schema) \
  .option("header", first_row_is_header) \
  .option("sep", delimiter) \
  .load(file_location)

display(df_ml)

_c0,event_type,product_id,category_id,brand,price,user_id,user_session,Year_Month,category,sub_category,product,purchase_or_not
3643316,view,100016625,2053013556630127071,fubag,46.05,616390077,463919c4-986c-4f3d-9b66-89294d767d07,2020-03-01T00:00:00.000+0000,construction,tools,drill,0
4088823,view,1004767,2232732093077520756,samsung,241.83,534804899,00a6dfd4-0c47-4afe-a350-b0376d992943,2020-03-01T00:00:00.000+0000,construction,tools,light,0
126172,view,5100853,2053013553341792533,apple,603.49,554727660,6ab020a2-2a93-4890-b33b-6a9137294828,2019-10-01T00:00:00.000+0000,electronics,clocks,clocks,0
557350,view,19200183,2053013556202308035,makita,235.76,513059829,cc73ed5a-5cee-41cf-ab5c-466d4bd64090,2019-10-01T00:00:00.000+0000,construction,tools,saw,0
939582,view,3601323,2053013563810775923,whirlpool,313.75,557678498,b27af875-98ac-4c65-b718-f39386a3ec06,2019-11-01T00:00:00.000+0000,appliances,kitchen,washer,0
5629444,view,100107882,2232732093077520756,huawei,331.51,541357021,1608368f-9588-4533-86f0-b1fd2799fd70,2020-04-01T00:00:00.000+0000,construction,tools,light,0
3427093,cart,1005161,2232732093077520756,xiaomi,164.95,596595074,b8499ccb-9154-44d6-8cc6-ca4c3e59e9f9,2020-02-01T00:00:00.000+0000,construction,tools,light,0
3117860,view,11200192,2232732105635267203,xiaomi,30.89,518039429,dedfa0eb-e552-4a27-86a3-d4dc9436782a,2020-02-01T00:00:00.000+0000,kids,swing,swing,0
1800196,view,1004849,2232732093077520756,huawei,884.49,600194504,5cdf46d8-bf52-49d8-82c9-3713166ff90f,2020-02-01T00:00:00.000+0000,construction,tools,light,0
313354,view,1004246,2053013555631882655,apple,734.81,515088375,22f561d0-e324-44fb-8214-18b952ea9bdd,2019-10-01T00:00:00.000+0000,electronics,smartphone,smartphone,0


In [0]:
# File location and type
file_location = "/mnt/s3data2/subset_7.csv"
file_type = "csv"

# CSV options
infer_schema = "true"
first_row_is_header = "true"
delimiter = ","

# The applied options are for CSV files. For other file types, these will be ignored.
subset7 = spark.read.format(file_type) \
  .option("inferSchema", infer_schema) \
  .option("header", first_row_is_header) \
  .option("sep", delimiter) \
  .load(file_location)

display(subset7)

_c0,event_type,product_id,category_id,brand,price,user_id,user_session,Year_Month,category,sub_category,product
0,view,3900821,2053013552326770905,aqua,33.2,554748717,9333dfbd-b87a-4708-9857-6336556b0fcc,2019-10-01T00:00:00.000+0000,appliances,environment,water_heater
2,view,1307067,2053013558920217191,lenovo,251.74,550050854,7c90fc70-0e80-4590-96f3-13c02c18c713,2019-10-01T00:00:00.000+0000,computers,notebook,notebook
3,view,1004237,2053013555631882655,apple,1081.98,535871217,c6bd7419-2748-4c56-95b4-8cec9ff8b80d,2019-10-01T00:00:00.000+0000,electronics,smartphone,smartphone
4,view,1480613,2053013561092866779,pulser,908.62,512742880,0d0d91c2-c9c2-4e81-90a5-86594dec0db9,2019-10-01T00:00:00.000+0000,computers,desktop,desktop
7,view,28719074,2053013565480109009,baden,102.71,520571932,ac1cd4e5-a3ce-4224-a2d7-ff660a105880,2019-10-01T00:00:00.000+0000,apparel,shoes,keds
8,view,1004545,2053013555631882655,huawei,566.01,537918940,406c46ed-90a4-4787-a43b-59a410c1a5fb,2019-10-01T00:00:00.000+0000,electronics,smartphone,smartphone
9,view,2900536,2053013554776244595,elenberg,51.46,555158050,b5bdd0b3-4ca2-4c55-939e-9ce44bb50abd,2019-10-01T00:00:00.000+0000,appliances,kitchen,microwave
10,view,1005011,2053013555631882655,samsung,900.64,530282093,50a293fb-5940-41b2-baf3-17af0e812101,2019-10-01T00:00:00.000+0000,electronics,smartphone,smartphone
11,view,3900746,2053013552326770905,haier,102.38,555444559,98b88fa0-d8fa-4b9d-8a71-3dd403afab85,2019-10-01T00:00:00.000+0000,appliances,environment,water_heater
13,view,13500240,2053013557099889147,brw,93.18,555446365,7f0062d8-ead0-4e0a-96f6-43a0b79a2fc4,2019-10-01T00:00:00.000+0000,furniture,bedroom,bed


In [0]:
%scala
import org.apache.hadoop.io.LongWritable
import org.apache.hadoop.io.Text
import org.apache.hadoop.conf.Configuration
import org.apache.hadoop.mapreduce.lib.input.TextInputFormat

val conf = new Configuration
conf.set("textinputformat.record.delimiter", "}")
val log_df = sc.newAPIHadoopFile("/mnt/MountName", classOf[TextInputFormat], classOf[LongWritable], classOf[Text], conf).map(_._2.toString)


import org.apache.hadoop.io.LongWritable
import org.apache.hadoop.io.Text
import org.apache.hadoop.conf.Configuration
import org.apache.hadoop.mapreduce.lib.input.TextInputFormat
conf: org.apache.hadoop.conf.Configuration = Configuration: core-default.xml, core-site.xml, mapred-default.xml, mapred-site.xml, yarn-default.xml, yarn-site.xml, hdfs-default.xml, hdfs-rbf-default.xml, hdfs-site.xml, hdfs-rbf-site.xml
log_df: org.apache.spark.rdd.RDD[String] = MapPartitionsRDD[45] at map at command-2961455995355518:8

### 2. Data Cleaning and Processing
In this section we will explore the structure of the data. We check to see if there are any anomalies present in the data, and deal with the null/missing values, if any.

In [0]:
#Checikng for null values in the dataset
df_1 = df1.select("brand", "price", "user_id", "event_type", "product_id", "category_id", "user_session", "category_code")
df_null = df_1.select([count(when(col(c).contains('None') | \
                            col(c).contains('NULL') | \
                            (col(c) == '' ) | \
                            col(c).isNull() | \
                            isnan(c), c 
                           )).alias(c)
                    for c in df_1.columns])
df_null.show()

+-------+-----+-------+----------+----------+-----------+------------+-------------+
|  brand|price|user_id|event_type|product_id|category_id|user_session|category_code|
+-------+-----+-------+----------+----------+-----------+------------+-------------+
|1007266|    0|      0|         0|         0|          0|           9|      1077958|
+-------+-----+-------+----------+----------+-----------+------------+-------------+



In [0]:
#Converting the dataframe to pandas dataframe
%python
python_df = pd.DataFrame()
python_df = df.toPandas()


/databricks/spark/python/pyspark/sql/pandas/conversion.py:210: UserWarning: toPandas attempted Arrow optimization because 'spark.sql.execution.arrow.pyspark.enabled' is set to true, but has reached the error below and can not continue. Note that 'spark.sql.execution.arrow.pyspark.fallback.enabled' does not have an effect on failures in the middle of computation.
  An error occurred while calling o624.getResult.
: org.apache.spark.SparkException: Exception thrown in awaitResult: 
	at org.apache.spark.util.ThreadUtils$.awaitResult(ThreadUtils.scala:429)
	at org.apache.spark.security.SocketAuthServer.getResult(SocketAuthServer.scala:107)
	at org.apache.spark.security.SocketAuthServer.getResult(SocketAuthServer.scala:103)
	at sun.reflect.NativeMethodAccessorImpl.invoke0(Native Method)
	at sun.reflect.NativeMethodAccessorImpl.invoke(NativeMethodAccessorImpl.java:62)
	at sun.reflect.DelegatingMethodAccessorImpl.invoke(DelegatingMethodAccessorImpl.java:43)
	at java.lang.reflect.Method.invoke(

---------------------------------------------------------------------------
Py4JJavaError                             Traceback (most recent call last)
<command-478676685993868> in <cell line: 2>()
      1 python_df = pd.DataFrame()
----> 2 python_df = df.toPandas()

/databricks/spark/python/pyspark/databricks/utils/instrumentation.py in wrapper(self, *args, **kwargs)
     41         try:
     42             start_time = time.time()
---> 43             return_val = func(self, *args, **kwargs)
     44         except Exception as e:
     45             duration = (time.time() - start_time) * 1000

/databricks/spark/python/pyspark/sql/pandas/conversion.py in toPandas(self)
    147                     tmp_column_names = ["col_{}".format(i) for i in range(len(self.columns))]
    148                     self_destruct = jconf.arrowPySparkSelfDestructEnabled()
--> 149                     batches = self.toDF(*tmp_column_names)._collect_as_arrow(
    150                         split_batches=sel

Checking the schema and data type for the different columns

In [0]:
%python
#Printing the schema
df.printSchema()

root
 |-- event_time: timestamp (nullable = true)
 |-- event_type: string (nullable = true)
 |-- product_id: integer (nullable = true)
 |-- category_id: long (nullable = true)
 |-- category_code: string (nullable = true)
 |-- brand: string (nullable = true)
 |-- price: double (nullable = true)
 |-- user_id: integer (nullable = true)
 |-- user_session: string (nullable = true)



In [0]:
#Printing the dataframe
df.show()

+-------------------+----------+----------+-------------------+--------------------+--------+-------+---------+--------------------+
|         event_time|event_type|product_id|        category_id|       category_code|   brand|  price|  user_id|        user_session|
+-------------------+----------+----------+-------------------+--------------------+--------+-------+---------+--------------------+
|2019-10-01 00:00:00|      view|  44600062|2103807459595387724|                null|shiseido|  35.79|541312140|72d76fde-8bb3-4e0...|
|2019-10-01 00:00:00|      view|   3900821|2053013552326770905|appliances.enviro...|    aqua|   33.2|554748717|9333dfbd-b87a-470...|
|2019-10-01 00:00:01|      view|  17200506|2053013559792632471|furniture.living_...|    null|  543.1|519107250|566511c2-e2e3-422...|
|2019-10-01 00:00:01|      view|   1307067|2053013558920217191|  computers.notebook|  lenovo| 251.74|550050854|7c90fc70-0e80-459...|
|2019-10-01 00:00:04|      view|   1004237|2053013555631882655|electr

It can be observed that the 'category_code' column has a data seperated by a dot('.'). We will be dividing the category_code column into 3 columns, category, sub-category and product. Furthermore, we will also be seperating the month and year from the timestamp to make it easier for analysis.

In [0]:
#Getting general overview of the data
df.select('event_type','price','category_code','brand' ).describe().show()

+-------+----------+------------------+-------------------+--------+
|summary|event_type|             price|      category_code|   brand|
+-------+----------+------------------+-------------------+--------+
|  count|  42448764|          42448764|           28933155|36335756|
|   mean|      null|290.32366068489216|               null|     NaN|
| stddev|      null| 358.2691553394021|               null|     NaN|
|    min|      cart|               0.0|    accessories.bag|  a-case|
|    max|      view|           2574.07|stationery.cartrige|   zyxel|
+-------+----------+------------------+-------------------+--------+



In [0]:
subset7.printSchema()

root
 |-- _c0: integer (nullable = true)
 |-- event_type: string (nullable = true)
 |-- product_id: integer (nullable = true)
 |-- category_id: long (nullable = true)
 |-- brand: string (nullable = true)
 |-- price: double (nullable = true)
 |-- user_id: integer (nullable = true)
 |-- user_session: string (nullable = true)
 |-- Year_Month: timestamp (nullable = true)
 |-- category: string (nullable = true)
 |-- sub_category: string (nullable = true)
 |-- product: string (nullable = true)



In [0]:
#Printing the dataframe
subset7.show()

+---+----------+----------+-------------------+--------+-------+---------+--------------------+-------------------+-----------+------------+------------+
|_c0|event_type|product_id|        category_id|   brand|  price|  user_id|        user_session|         Year_Month|   category|sub_category|     product|
+---+----------+----------+-------------------+--------+-------+---------+--------------------+-------------------+-----------+------------+------------+
|  0|      view|   3900821|2053013552326770905|    aqua|   33.2|554748717|9333dfbd-b87a-470...|2019-10-01 00:00:00| appliances| environment|water_heater|
|  2|      view|   1307067|2053013558920217191|  lenovo| 251.74|550050854|7c90fc70-0e80-459...|2019-10-01 00:00:00|  computers|    notebook|    notebook|
|  3|      view|   1004237|2053013555631882655|   apple|1081.98|535871217|c6bd7419-2748-4c5...|2019-10-01 00:00:00|electronics|  smartphone|  smartphone|
|  4|      view|   1480613|2053013561092866779|  pulser| 908.62|512742880|0d

In [0]:
#Getting general overview of the data
subset7.select('event_type','price','category','sub_category','product','brand' ).describe().show()

+-------+----------+------------------+-----------+------------+--------+--------+
|summary|event_type|             price|   category|sub_category| product|   brand|
+-------+----------+------------------+-----------+------------+--------+--------+
|  count|   6021400|           6021400|    6021400|     6021400| 6021400| 6021400|
|   mean|      null|328.44094459761135|       null|        null|    null|Infinity|
| stddev|      null| 375.7939840182123|       null|        null|    null|     NaN|
|    min|      cart|              0.77|accessories| accessories|acoustic|  a-case|
|    max|      view|           2574.07| stationery|      wallet|   winch|   zyxel|
+-------+----------+------------------+-----------+------------+--------+--------+



In [0]:
#Checikng for null values in the dataset
df_1 = subset7.select("brand", "price", "user_id", "event_type", "product_id", "category_id", "user_session", "category","sub_category","product" )
df_null = df_1.select([count(when(col(c).contains('None') | \
                            col(c).contains('NULL') | \
                            (col(c) == '' ) | \
                            col(c).isNull() | \
                            isnan(c), c 
                           )).alias(c)
                    for c in df_1.columns])
df_null.show()

+-----+-----+-------+----------+----------+-----------+------------+--------+------------+-------+
|brand|price|user_id|event_type|product_id|category_id|user_session|category|sub_category|product|
+-----+-----+-------+----------+----------+-----------+------------+--------+------------+-------+
|    0|    0|      0|         0|         0|          0|           0|       0|           0|      0|
+-----+-----+-------+----------+----------+-----------+------------+--------+------------+-------+



### 3. Data Exploration
Category_code is divided into 3 columns, category, sub_category, and product. The year and month is extracted from the timestamp to make it easier for analysis.

In [0]:
subset7.show()

+---+----------+----------+-------------------+--------+-------+---------+--------------------+-------------------+-----------+------------+------------+
|_c0|event_type|product_id|        category_id|   brand|  price|  user_id|        user_session|         Year_Month|   category|sub_category|     product|
+---+----------+----------+-------------------+--------+-------+---------+--------------------+-------------------+-----------+------------+------------+
|  0|      view|   3900821|2053013552326770905|    aqua|   33.2|554748717|9333dfbd-b87a-470...|2019-10-01 00:00:00| appliances| environment|water_heater|
|  1|      view|  17200506|2053013559792632471| Unknown|  543.1|519107250|566511c2-e2e3-422...|2019-10-01 00:00:00|  furniture| living_room|        sofa|
|  2|      view|   1307067|2053013558920217191|  lenovo| 251.74|550050854|7c90fc70-0e80-459...|2019-10-01 00:00:00|  computers|    notebook|    notebook|
|  3|      view|   1004237|2053013555631882655|   apple|1081.98|535871217|c6

##### Finding the distribution of the price column

In [0]:
#Distributing prices into quarties
quantile = subset7.approxQuantile(['price'], [0.25, 0.5, 0.75], 0)
quantile_25 = quantile[0][0]
quantile_50 = quantile[0][1]
quantile_75 = quantile[0][2]
#Printing the values
print('quantile_25: '+str(quantile_25))
print('quantile_50: '+str(quantile_50))
print('quantile_75: '+str(quantile_75))

quantile_25: 87.43
quantile_50: 193.75
quantile_75: 411.82


It can be observed that 50% of the prices fall between $87.43 and $411.82. This is a significant difference and we can assume that this is because the website sells a variety of product categories.

In [0]:
#Getting general overview of the data
subset7.select('event_type','price','category','sub_category','product','brand' ).describe().show()

+-------+----------+------------------+-----------+------------+--------+--------+
|summary|event_type|             price|   category|sub_category| product|   brand|
+-------+----------+------------------+-----------+------------+--------+--------+
|  count|   6021400|           6021400|    6021400|     6021400| 6021400| 6021400|
|   mean|      null|328.44094459761135|       null|        null|    null|Infinity|
| stddev|      null| 375.7939840182123|       null|        null|    null|     NaN|
|    min|      cart|              0.77|accessories| accessories|acoustic|  a-case|
|    max|      view|           2574.07| stationery|      wallet|   winch|   zyxel|
+-------+----------+------------------+-----------+------------+--------+--------+



Creating a sales dataframe where event type is purchase to analyze the sales data.

In [0]:
#Creating a dataframe of products sold
sales = subset7.filter(col("event_type") == "purchase")

In [0]:
# showing top 10 brands by revenue generated
top_brands_sale_price = sales.select("brand", "price").groupBy("brand").sum()
top_brands = top_brands_sale_price.orderBy("sum(price)", ascending=False).show(10)
top_brands

+-------+--------------------+
|  brand|          sum(price)|
+-------+--------------------+
|  apple|1.6677616739999928E7|
|samsung|   8396984.610000027|
| xiaomi|  1581966.9000000036|
|   acer|  1163081.9600000056|
| huawei|   881652.2499999978|
|     lg|           698584.82|
| lenovo|   691093.3500000004|
|   asus|   609585.1000000002|
|   oppo|   601915.2499999994|
|     hp|   526364.1099999989|
+-------+--------------------+
only showing top 10 rows



All of the top 10 brands by revenue generated are electronic brands. We can assume that the website has the highest revenue generated by the sale of electronics.

In [0]:
# showing top 10 products by revenue generated
top_product_sale_price = sales.select("product", "price").groupBy("product").sum()
top_product = top_product_sale_price.orderBy("sum(price)", ascending=False).show(10)
top_product

+-------------+--------------------+
|      product|          sum(price)|
+-------------+--------------------+
|        light|1.7248878310000498E7|
|   smartphone|   7605629.930000209|
|    headphone|  3161496.3100000084|
|     massager|   1244865.760000001|
|refrigerators|   981959.6900000003|
|       clocks|   681612.6400000007|
|       washer|   616407.1200000003|
|      bicycle|  476702.95999999996|
|     notebook|  465017.38000000006|
|        shoes|  431296.25000000023|
+-------------+--------------------+
only showing top 10 rows



We can see that light is the product with the highest revenue generated. The next few products followed by light are electronic appliances, which support the claim that the website generates majority of the revenue through electronics and are the strongest in that market segment. We will analyze the light product later in this notebook.

In [0]:
# showing top 10 categories by revenue generated
top_cat_sale_price = sales.select("category", "price").groupBy("category").sum()
top_cat = top_cat_sale_price.orderBy("sum(price)", ascending=False).show(10)
top_cat

+------------+--------------------+
|    category|          sum(price)|
+------------+--------------------+
|construction|  1.74071817300005E7|
| electronics|1.2048650600000387E7|
|  appliances|   3995096.129999997|
|     apparel|  1051910.6499999978|
|   computers|   814492.0999999996|
|       sport|   572447.4600000005|
|   furniture|  298627.20999999956|
|        kids|  189064.36999999985|
|        auto|   86130.36999999997|
| accessories|            25724.91|
+------------+--------------------+
only showing top 10 rows



It can be observed that construction category generates the highest revenue, followed by electronics and appliances. This will be analyzed in the next part of the notebook.

In [0]:
# showing top 10 sub_categories by revenue generated
top_subcat_sale_price = subset7.select("sub_category", "price").groupBy("sub_category").sum()
top_subcat = top_subcat_sale_price.orderBy("sum(price)", ascending=False).show(10)
top_subcat

+------------+--------------------+
|sub_category|          sum(price)|
+------------+--------------------+
|       tools| 6.855185210105314E8|
|  smartphone| 2.698721960299141E8|
|       audio|2.4959878669003406E8|
|     kitchen|2.1689654099000844E8|
|    personal| 7.769442935000719E7|
|       shoes| 7.536958850999294E7|
|      clocks| 5.946499492000162E7|
|    notebook| 5.303130959000351E7|
| environment| 4.067496874999921E7|
|       video|3.1635165779998727E7|
+------------+--------------------+
only showing top 10 rows



Tools having the highest revenue generated might be attributed to the fact that light is categorized as a tool in some rows and light has the highest revenue generated.

### 4. Data Visualization and Analysis

#### 1. Distribution for event type
Finding the conversion rate of the website for the number of people viewing, adding to the cart and buying. If the number of people adding to the cart is much higher than the number of people buying the product, that may signify that the customers face issues placing orders or making payments.

In [0]:
#Displaying the distribution for event type
display(subset7.select("event_type"))

event_type
view
view
view
view
view
view
view
view
view
view


Output can only be rendered in Databricks

The ratio of people putting things in the cart and purchasing products is good. This shows that the website is working fine.

#### 2. Top 10 by revenue in each category

##### a. Top 10 brands by revenue
Finding the brands which generate the most revenue for the website.

In [0]:
#Printing the top 10 brands by revenue
sales = sales.na.drop()
df5 = sales.select("brand", "price")
df5 = df5.na.drop()
rolled1 = df5.rollup( "brand").sum().sort("brand")
rolled1 = rolled1.na.drop()
display(rolled1.sort(col("sum(price)").desc()).head(10))

brand,sum(price)
apple,1.6677616739999928E7
samsung,8396984.610000027
xiaomi,1581966.9000000036
acer,1163081.9600000056
huawei,881652.2499999978
lg,698584.82
lenovo,691093.3500000004
asus,609585.1000000002
oppo,601915.2499999994
hp,526364.1099999989


Output can only be rendered in Databricks

The brands which generate the highest revenue are all electronic and tech brands. The website generates a major chunk of its revenue from electronics.

##### b. Top 10 categories by revenue
Finding the categories which generate the most revenue for the website

In [0]:
#Printing the top 10 categories by revenue
sales = sales.na.drop()
df5 = sales.select("category", "price")
df5 = df5.na.drop()
rolled1 = df5.rollup( "category").sum().sort("category")
rolled1 = rolled1.na.drop()
display(rolled1.sort(col("sum(price)").desc()).head(10))

category,sum(price)
construction,1.74071817300005E7
electronics,1.2048650600000387E7
appliances,3995096.129999997
apparel,1051910.6499999978
computers,814492.0999999996
sport,572447.4600000005
furniture,298627.20999999956
kids,189064.36999999985
auto,86130.36999999997
accessories,25724.91


Output can only be rendered in Databricks

Construction has the highest revenue generated, followed by electronics and appliances. We will further analyze why construction has the highest revenue generated.

##### c. Top 10 sub-categories by revenue
Finding the sub-categories which generate the most revenue for the website

In [0]:
#Printing the top 10 sub-categories by revenue
sales = sales.na.drop()
df5 = sales.select("sub_category", "price")
df5 = df5.na.drop()
rolled1 = df5.rollup( "sub_category").sum().sort("sub_category")
rolled1 = rolled1.na.drop()
display(rolled1.sort(col("sum(price)").desc()).head(10))

sub_category,sum(price)
tools,1.7349160560000505E7
smartphone,7605629.930000209
audio,3232078.4300000067
kitchen,2256764.839999994
personal,1257427.9200000013
shoes,859846.5899999994
clocks,681612.6400000007
bicycle,476702.95999999996
notebook,465017.38000000006
video,414870.5700000003


Output can only be rendered in Databricks

Tools having the highest revenue generated might be attributed to the fact that light is categorized as a tool in some rows and light has the highest revenue generated.

##### d. Top 10 products by revenue
Finding the products which generate the most revenue for the website

In [0]:
#Printing the top 10 products by revenue
sales = sales.na.drop()
df5 = sales.select("product", "price")
df5 = df5.na.drop()
rolled1 = df5.rollup( "product").sum().sort("product")
rolled1 = rolled1.na.drop()
display(rolled1.sort(col("sum(price)").desc()).head(10))

product,sum(price)
light,1.7248878310000498E7
smartphone,7605629.930000209
headphone,3161496.3100000084
massager,1244865.760000001
refrigerators,981959.6900000003
clocks,681612.6400000007
washer,616407.1200000003
bicycle,476702.95999999996
notebook,465017.38000000006
shoes,431296.25000000023


Output can only be rendered in Databricks

Light generates more than twice of the renevue generated by smartphones. Most of the products following light are electronic products.

#### 3. Analysis of the top 5 brands
We will analyze the top 5 brands on the website and find out if there are any trends or patterns present. We will analyze the products they sell, the categories and the event type trends for these brands.

In [0]:
#Filtering on brands on the entire dataset
apple_all = subset7.filter(col("brand") == "apple")
lenovo_all = subset7.filter(col("brand") == "lenovo")
huawei_all = subset7.filter(col("brand") == "huawei")
samsung_all = subset7.filter(col("brand") == "samsung")
xiaomi_all = subset7.filter(col("brand") == "xiaomi")
acer_all = subset7.filter(col("brand") == "acer")

#Filtering on brands for the purchase dataset
apple_sales = sales.filter(col("brand") == "apple")
lenovo_sales = sales.filter(col("brand") == "lenovo")
huawei_sales = sales.filter(col("brand") == "huawei")
samsung_sales = sales.filter(col("brand") == "samsung")
xiaomi_sales = sales.filter(col("brand") == "xiaomi")
acer_sales = sales.filter(col("brand") == "acer")

##### a. Finding the distribution for event type for each brand

In [0]:
#Distribution of event type for apple
display(apple_all.select("event_type"))

event_type
view
view
view
view
view
view
view
view
view
view


Output can only be rendered in Databricks

In [0]:
#Distribution of event type for samsung
display(samsung_all.select("event_type"))

event_type
view
view
view
view
view
view
view
view
view
view


Output can only be rendered in Databricks

In [0]:
#Distribution of event type for xiaomi
display(xiaomi_all.select("event_type"))

event_type
view
view
view
view
view
view
view
view
view
view


Output can only be rendered in Databricks

In [0]:
#Distribution of event type for huawei
display(huawei_all.select("event_type"))

event_type
view
view
view
view
view
view
view
view
view
view


Output can only be rendered in Databricks

In [0]:
#Distribution of event type for acer
display(acer_all.select("event_type"))

event_type
view
view
view
view
view
view
view
view
view
view


Output can only be rendered in Databricks

It is observed that the converstion ratio of items being put in cart to items being purchased is lower for expensive brands like Apple and Samsung as compared to Huawei and Acer. This might be because people deliberate a lot before buying expensive products.

In [0]:
#Creating a dataframe of top 5 brands
import pyspark.sql.functions as f
df_top5_sales = sales.filter((f.col('brand') == 'apple')| (f.col('brand') == 'samsung')| (f.col('brand') == 'xiaomi')| (f.col('brand') == 'acer')| (f.col('brand') == 'huawei'))
df_top5_sales.show()

+----+----------+----------+-------------------+-------+-------+---------+--------------------+-------------------+-----------+------------+----------+
| _c0|event_type|product_id|        category_id|  brand|  price|  user_id|        user_session|         Year_Month|   category|sub_category|   product|
+----+----------+----------+-------------------+-------+-------+---------+--------------------+-------------------+-----------+------------+----------+
| 161|  purchase|   1004856|2053013555631882655|samsung| 130.76|543272936|8187d148-3c41-46d...|2019-10-01 00:00:00|electronics|  smartphone|smartphone|
| 307|  purchase|   1002532|2053013555631882655|  apple| 642.69|551377651|3c80f0d6-e9ec-418...|2019-10-01 00:00:00|electronics|  smartphone|smartphone|
| 573|  purchase|   4804055|2053013554658804075|  apple| 189.91|524601178|2af9b570-0942-4dc...|2019-10-01 00:00:00|electronics|       audio| headphone|
| 602|  purchase|   4804056|2053013554658804075|  apple| 161.98|551377651|3c80f0d6-e9ec-

##### b. Products generating highest revenue for top 5 brands

In [0]:
#Products generating the highest revenue
df5 = df_top5_sales.select("product", "price")
df5 = df5.na.drop()
rolled1 = df5.rollup( "product").sum().sort("product")
rolled1 = rolled1.na.drop()
display(rolled1.sort(col("sum(price)").desc()).head(10))

product,sum(price)
light,1.6531775960000329E7
smartphone,7423527.860000197
headphone,1614664.9400000023
massager,615501.1200000003
clocks,538510.5700000004
bicycle,426014.91999999946
refrigerators,359788.63000000006
slipons,260884.25999999992
notebook,242224.7399999999
washer,217737.5700000001


Output can only be rendered in Databricks

We can observe that the top 10 products contains bicycle. The top 5 brands are all tech brands, and these products might have been classified incorrectly.

##### c. Analyzing products classified as 'Light'

In [0]:
#Creating a dataframe with only light as product
light_sales = sales.filter(col("product") == "light")

In [0]:
#Top 10 brands by revenue for light
df5 = light_sales.select("brand", "price")
df5 = df5.na.drop()
rolled1 = df5.rollup( "brand").sum().sort("brand")
rolled1 = rolled1.na.drop()
display(rolled1.sort(col("sum(price)").desc()).head(10))

brand,sum(price)
apple,9974850.690000007
samsung,4913063.710000064
xiaomi,1030635.9200000002
huawei,613225.6399999984
oppo,496536.2999999993
vivo,73773.24
meizu,25956.48999999999
oneplus,25289.369999999995
realme,18472.35
honor,16746.49


Output can only be rendered in Databricks

On finding the top brands for light, we can see that apple comes to be the top brand and it is followed by samsung. It might be the case that the products on the website are misclassified.

In [0]:
#Top categories for light
df5 = light_sales.select("category", "price")
df5 = df5.na.drop()
rolled1 = df5.rollup( "category").sum().sort("category")
rolled1 = rolled1.na.drop()
display(rolled1.sort(col("sum(price)").desc()).head(10))

category,sum(price)
construction,1.72474160300005E7
furniture,1462.28


Lights are correctly classified on category, but might be incorrectly classified on brands. Lights might have been classified as construction and furniture as they are used indoor and in this case they might be referring to lights used on construction sites.

In [0]:
#Top sub-categories for light
df5 = light_sales.select("sub_category", "price")
df5 = df5.na.drop()
rolled1 = df5.rollup( "sub_category").sum().sort("sub_category")
rolled1 = rolled1.na.drop()
display(rolled1.sort(col("sum(price)").desc()).head(10))

sub_category,sum(price)
tools,1.72474160300005E7
universal,1462.28


Light used in construction are classified as tools.

Upon observing key products that drive sales for the e-commerce website, we find that construction products, electronic appliances, and apparels naturally drive more sales. A deeper look into category specifics suggests that home lights dominate the construction category. Smartphones, headphones, and massagers are hot sellers in the electronics category.


Apple and Samsung are the best-selling brands on the website, complementing the evidence that smartphones also happen to be among the top-selling product sub-categories. A brand such as Samsung, however, could be selling multiple products under the same brand. Hence it could be useful to understand the best-selling products for this brand and boost revenue.

It can be observed that the product 'light' might have been categorized incorrectly for brands. The top brands present for 'light' are Apple, Samsung, Xiaomi, Huawei, and Oppo. The category for these products are construction and furniture, and the brands mentioned above are majorly tech companies.

### 5. Machine Learning
We will train a model to predict if a particular product is most likely to be purchased or not. This would help the website to stack up the product and keep it read, or put certain products on sale so they attract more customers. 

We have created a column which indicates if the item was purchased or not. We found that our data was highly imbalanced with not purchased being the majority class and purchased being the minority class. The minority class has been oversampled and the majority class has been undersampled in the dataset used.

In [0]:
#Viewing the dataset
df_ml.show()

+-------+----------+----------+-------------------+---------+------+---------+--------------------+-------------------+------------+------------+-------------+---------------+
|    _c0|event_type|product_id|        category_id|    brand| price|  user_id|        user_session|         Year_Month|    category|sub_category|      product|purchase_or_not|
+-------+----------+----------+-------------------+---------+------+---------+--------------------+-------------------+------------+------------+-------------+---------------+
|3643316|      view| 100016625|2053013556630127071|    fubag| 46.05|616390077|463919c4-986c-4f3...|2020-03-01 00:00:00|construction|       tools|        drill|              0|
|4088823|      view|   1004767|2232732093077520756|  samsung|241.83|534804899|00a6dfd4-0c47-4af...|2020-03-01 00:00:00|construction|       tools|        light|              0|
| 126172|      view|   5100853|2053013553341792533|    apple|603.49|554727660|6ab020a2-2a93-489...|2019-10-01 00:00:00| 

In [0]:
#Viewing the schema
df_ml.printSchema()

root
 |-- _c0: integer (nullable = true)
 |-- event_type: string (nullable = true)
 |-- product_id: integer (nullable = true)
 |-- category_id: long (nullable = true)
 |-- brand: string (nullable = true)
 |-- price: double (nullable = true)
 |-- user_id: integer (nullable = true)
 |-- user_session: string (nullable = true)
 |-- Year_Month: timestamp (nullable = true)
 |-- category: string (nullable = true)
 |-- sub_category: string (nullable = true)
 |-- product: string (nullable = true)
 |-- purchase_or_not: integer (nullable = true)



In [0]:
#Selecting columns useful for the model
df_model = df_ml.select('brand', 'price', 'Year_Month', 'category', 'sub_category', 'product','purchase_or_not')
cols = df_model.columns
df_model.printSchema()

root
 |-- brand: string (nullable = true)
 |-- price: double (nullable = true)
 |-- Year_Month: timestamp (nullable = true)
 |-- category: string (nullable = true)
 |-- sub_category: string (nullable = true)
 |-- product: string (nullable = true)
 |-- purchase_or_not: integer (nullable = true)



##### Preparing the data for machine learning.
The data contains categorical variable and these cariables have to be converted into 1s and 0s before we create the model.
The code below is taken from databricks’ official site and it indexes each categorical column using the StringIndexer, then converts the indexed categories into one-hot encoded variables i.e. dummy variables. The VectorAssembler then converts these multiple columns into one vectorized column.

In [0]:
#Creating dummies and a feature column with dummies
from pyspark.ml.feature import OneHotEncoder,StringIndexer, VectorAssembler
categoricalColumns = [ 'brand', 'sub_category', 'product']
stages = []
for categoricalCol in categoricalColumns:
    stringIndexer = StringIndexer(inputCol = categoricalCol, outputCol = categoricalCol + 'Index')
    encoder = OneHotEncoder(inputCols=[stringIndexer.getOutputCol()], outputCols=[categoricalCol + "classVec"])
    stages += [stringIndexer, encoder]
label_stringIdx = StringIndexer(inputCol = 'purchase_or_not', outputCol = 'label')
stages += [label_stringIdx]
numericCols = ['price']
assemblerInputs = [c + "classVec" for c in categoricalColumns] + numericCols
assembler = VectorAssembler(inputCols=assemblerInputs, outputCol="features")
stages += [assembler]

In [0]:
from pyspark.ml import Pipeline
pipeline = Pipeline(stages = stages)
pipelineModel = pipeline.fit(df_model)
df_model = pipelineModel.transform(df_model)
selectedCols = ['label', 'features'] + cols
df_model = df_model.select(selectedCols)
df_model.printSchema()

root
 |-- label: double (nullable = false)
 |-- features: vector (nullable = true)
 |-- brand: string (nullable = true)
 |-- price: double (nullable = true)
 |-- Year_Month: timestamp (nullable = true)
 |-- category: string (nullable = true)
 |-- sub_category: string (nullable = true)
 |-- product: string (nullable = true)
 |-- purchase_or_not: integer (nullable = true)



In [0]:
#Viewing dataset
pd.DataFrame(df_model.take(5), columns=df_model.columns).transpose()

,0,1,2,3,4
label,1.0,1.0,1.0,1.0,1.0
features,"(0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","(1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","(0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","(0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","(0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
brand,fubag,samsung,apple,makita,whirlpool
price,46.05,241.83,603.49,235.76,313.75
Year_Month,2020-03-01 00:00:00,2020-03-01 00:00:00,2019-10-01 00:00:00,2019-10-01 00:00:00,2019-11-01 00:00:00
category,construction,construction,electronics,construction,appliances
sub_category,tools,tools,clocks,tools,kitchen
product,drill,light,clocks,saw,washer
purchase_or_not,0,0,0,0,0


We can see a features column containing a vector of 1s and 0s.

Randomly splitting the dataset into 80-20 for train and test respectively.

In [0]:
#Splitting the dataset
train, test = df_model.randomSplit([0.8, 0.2])

#Couting the number of rows in each
print("Training Dataset Count: " + str(train.count()))
print("Test Dataset Count: " + str(test.count()))

Training Dataset Count: 3279841
Test Dataset Count: 820489


##### Logistic Regression

In [0]:
#Fitting the Logistic Regression model
from pyspark.ml.classification import LogisticRegression
lr = LogisticRegression(featuresCol = 'features', labelCol = 'label', maxIter=10)
lrModel = lr.fit(train)

In [0]:
#Making predictions
predictions = lrModel.transform(test)

In [0]:
#Checking the accuracy
from pyspark.ml.evaluation import MulticlassClassificationEvaluator, BinaryClassificationEvaluator
evaluator=MulticlassClassificationEvaluator(labelCol = "label", metricName="accuracy")
print('Accuracy', evaluator.evaluate(predictions))

Accuracy 0.5926136730656962


##### Decision Tree classifier

In [0]:
#Fitting the Decision Tree classifier model
from pyspark.ml.classification import DecisionTreeClassifier
dt = DecisionTreeClassifier(featuresCol = 'features', labelCol = 'label', maxDepth = 3)
dtModel = dt.fit(train)

#Making predictions
predictions = dtModel.transform(test)

In [0]:
#Checking the accuracy
from pyspark.ml.evaluation import MulticlassClassificationEvaluator, BinaryClassificationEvaluator
evaluator=MulticlassClassificationEvaluator(labelCol = "label", metricName="accuracy")
print('Accuracy', evaluator.evaluate(predictions))

Accuracy 0.573222935745141


##### Random Forest Classifier

In [0]:
#Fitting the Random Forest classifier model
from pyspark.ml.classification import RandomForestClassifier
rf = RandomForestClassifier(featuresCol = 'features', labelCol = 'label')
rfModel = rf.fit(train)

#Making predictions
predictions = rfModel.transform(test)

In [0]:
#Checking the accuracy
evaluator=MulticlassClassificationEvaluator(labelCol = "label", metricName="accuracy")
print('Accuracy', evaluator.evaluate(predictions))

Accuracy 0.561400579410571


##### Gradient Boosted Tree

In [0]:
#Fitting the gradient boosted tree classifier
from pyspark.ml.classification import GBTClassifier
gbt = GBTClassifier(maxIter=10)
gbtModel = gbt.fit(train)

#Making predictions
predictions = gbtModel.transform(test)


In [0]:
#Checking the accuracy
evaluator=MulticlassClassificationEvaluator(labelCol = "label", metricName="accuracy")
print('Accuracy', evaluator.evaluate(predictions))


Accuracy 0.5871937344681037


We can observe that Logistic regression and Gradient Boosted tree have the highest accuracy compared to the other two models. Although the accuracy is not that high, it would be useful to take calculated risk on certain products. It could help decide which products to stock up and which products to put on sale for stock clearance.

### 6. Conculsion

This study mainly focuses on the factors that affect consumers' online shopping behaviors and examines those factors to get an understanding of how users interact with the website. We use a dataset of multicategory online stores to perform Exploratory Data Analysis and Predictive Modeling. Our analysis helped us get insights like the most consumed product and category of all, and the top brands that generate the most revenue and conversion rate of customers. Since these factors link directly to profitability and revenue, it plays a crucial role in corporate planning. It’s helpful in determining the elements in a category that influence a buyer to select one product over another.

The model helped in knowing the likelihood of whether the product will sell or not and make business decisions regaring stocking up and introducing sales. 


The website has major traffic looking for electronics appliances, this might be because they offer good discounts on electronic products specifically or have good return policies and service for these products. The website should focus on this category moving forward as its their strong point.